In [1]:
import gzip
from contextlib import closing
from xml.etree import ElementTree as etree
from bs4 import BeautifulSoup
import os
from contextlib import closing
from xml.etree import ElementTree as etree
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from six.moves import cPickle


In [2]:
news_dataset = "/datasets/dk/news-short-2015-2"

keywords = ["farmers","draught","water scarcity","drought","hosepipe ban","water scarcity","water conservation",
            "water crisis" , "water resilience" , "water reuse" , "shower ban", "water use","water consumption"
            "water saving" , "shower" ,"shower less", "less water", "bathing less","irrigation" , "crop yield" ,
            "dry shampoo", "grey water", "bathing wipes" ,"farmer issues" , "lower yield","crops" ,"hot summer",
            "scorching" , "heat wave", "water cuts" , "watercuts" , "water shortage" , "dry summer", "hot weather", 
            "picnic weather", "beach weather","blocking", "blocking anticyclone", "anticyclone" , "el nino",
            "El Nino", "La Nina"]
total = 0
finalData = "/datasets/sagarj/water_scarcity/waterNewsDataMatched_2015_2.pkl"
curatedData = []

currentDir =  os.getcwd();
logPickle = currentDir + "/curationMatched_2015_2.pk"

In [3]:
def walkLevel1Dir(root):
    count = 0
    dirList = []
    filesList = []
    for path, dirs, files in os.walk(root):
        print path
        if count > 0:
            return dirList , fileList
        dirList = dirs
        fileList = files
        count = count + 1
        
def walkDir(root):
    f = []
    for (dirpath, dirnames, filenames) in os.walk(root):
        f.extend(filenames)
        break
    return filenames

In [4]:
files_big = walkDir(news_dataset)
files = []
for i in range(len(files_big)):
    if i%8 == 0:
        files.append(files_big[i])
#print files
total = float(len(files))


In [ ]:
count = 0.0
for f in files:
    path = news_dataset + "/" + f
    print "Parsing %s" % path
    
    with gzip.open(path) as archive:
        articles = []
        fdata=archive.read()
        xmldata=BeautifulSoup(fdata)
        articles =  xmldata.html.body.findAll('article')
        
        for article in articles:
            line =  article.find('body-cleartext').text
            matched = 0
            matched_strings=[]
            
            for keyword in keywords:
                if(line.find(keyword) > 0):
                    matched = matched + 1
                    matched_strings.append(keyword)
            
            if (matched > 0):
                country = ""
                lat = ""
                log = ""
                if article.location is not None and article.location.country is not None  : 
                    country = article.location.country.text
                if article.location is not None and article.location.latitude is not None:
                    lat = article.location.latitude.text
                if article.location is not None and article.location.longitude is not None :
                    log = article.location.longitude.text
                
                data={
                    'country' :  country ,
                    'latitude' : lat ,
                    'longitude' : log ,
                    'timeStamp' : article.find('retrieved-date').text ,
                    'text' : line,
                    'matched' : matched_strings
                    }
                curatedData.append(data)
    
    count = count + 1
    done = (count/total) * 100
    logline = "Progress %f percent"%(done)
    print logline
    flog = open(logPickle, 'a+')
    cPickle.dump(logline , flog);
    flog.close()
            
    

Parsing /datasets/dk/news-short-2015-2/public-news-2015-05-08T21-07-23Z.xml.gz
Progress 0.007955 percent
Parsing /datasets/dk/news-short-2015-2/public-news-2015-06-17T08-32-21Z.xml.gz
Progress 0.015911 percent
Parsing /datasets/dk/news-short-2015-2/public-news-2015-06-01T17-03-52Z.xml.gz
Progress 0.023866 percent
Parsing /datasets/dk/news-short-2015-2/public-news-2015-07-12T18-27-44Z.xml.gz
Progress 0.031822 percent
Parsing /datasets/dk/news-short-2015-2/public-news-2015-05-15T17-17-45Z.xml.gz
Progress 0.039777 percent
Parsing /datasets/dk/news-short-2015-2/public-news-2015-04-02T12-52-28Z.xml.gz
Progress 0.047733 percent
Parsing /datasets/dk/news-short-2015-2/public-news-2015-04-21T17-45-02Z.xml.gz
Progress 0.055688 percent
Parsing /datasets/dk/news-short-2015-2/public-news-2015-06-10T16-39-07Z.xml.gz
Progress 0.063644 percent
Parsing /datasets/dk/news-short-2015-2/public-news-2015-05-11T17-59-16Z.xml.gz
Progress 0.071599 percent
Parsing /datasets/dk/news-short-2015-2/public-news-2015

In [ ]:
f = open(finalData, 'a+')
cPickle.dump(curatedData, f, protocol=cPickle.HIGHEST_PROTOCOL)
f.close()

In [ ]:
import cPickle as pickle
with open('data_2.p', 'wb') as fp:
    pickle.dump(curatedData, fp)

In [ ]:
import json
with open('curated_2.json', 'w') as fp:
    json.dump(curatedData, fp)